In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '11_1'

In [3]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = '/content/drive/MyDrive/MGU/outputs/color/'
    gray_imgs = '/content/drive/MyDrive/MGU/outputs/gray/'
    checkpoints = '/content/drive/MyDrive/MGU/checkpoints'
elif kaggle:
    os.makedirs(test_number, exist_ok=True)
    results = "results"
    os.makedirs(results, exist_ok=True)
    dataset = '/kaggle/input/cifar10/'
else:
    dataset = '../../datasets/cifar10/'

In [4]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [5]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [6]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [7]:
SIZE = 32
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.ToTensor(),
                transforms.Normalize((0.4918019, 0.48263696, 0.44733968), (0.24705184, 0.2433789, 0.26162848)),
                transforms.ToPILImage(),
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
                transforms.ToTensor(),
                transforms.Normalize((0.4918019, 0.48263696, 0.44733968), (0.24705184, 0.2433789, 0.26162848)),
                transforms.ToPILImage(),
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [8]:
# Training
batch_size = 128
train_imagefolder = LabImageFolder(dataset+'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset+'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [9]:
kernel_size=3
stride_en=2
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 64

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        
        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm2 = nn.BatchNorm2d(channels_base)
        self.batchnorm3 = nn.BatchNorm2d(channels_base * 2)
        
        
    def forward(self, input):
        # encoder
        x = F.relu(self.batchnorm2(self.conv1(input)))
        x = F.relu(self.batchnorm3(self.conv2(x)))
        
        # decoder
        x = F.relu(self.batchnorm2(self.convtrans1(x)))
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.relu(self.batchnorm1(self.convtrans2(x)))
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [10]:
model = Autoencoder()

In [11]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [13]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [14]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]             640
       BatchNorm2d-2           [-1, 64, 16, 16]             128
            Conv2d-3            [-1, 128, 8, 8]          73,856
       BatchNorm2d-4            [-1, 128, 8, 8]             256
   ConvTranspose2d-5             [-1, 64, 8, 8]          73,792
       BatchNorm2d-6             [-1, 64, 8, 8]             128
   ConvTranspose2d-7           [-1, 32, 16, 16]          18,464
       BatchNorm2d-8           [-1, 32, 16, 16]              64
   ConvTranspose2d-9            [-1, 2, 16, 16]             578
Total params: 167,906
Trainable params: 167,906
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.57
Params size (MB): 0.64
Estimated Total Size (MB): 1.21
-------------------------------------------

In [15]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [16]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:        
#         writer.add_scalars(f'loss/test', {
#             'MSE': _loss[0].avg,
#             'PSNR': _loss[1].avg,
#             'SSIM': _loss[2].avg,
#         }, epoch)
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [17]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
#         writer.add_scalars(f'loss/train', {
#             'MSE': _loss[0].avg,
#             'PSNR': _loss[1].avg,
#             'SSIM': _loss[2].avg,
#         }, epoch)
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [18]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.02512388 (0.06097265), PSNR 15.99913120 (14.92869398), SSIM 0.10445468 (0.08515537)
Finished training epoch 0
Validate: MSE 0.02428684 (0.02587752), PSNR 16.14628792 (15.87720569), SSIM 0.06893195 (0.09365179)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.02496586 (0.02488198), PSNR 16.02653503 (16.04388313), SSIM 0.10746951 (0.11034812)
Finished training epoch 1
Validate: MSE 0.02369463 (0.02528477), PSNR 16.25349998 (15.97780598), SSIM 0.07036954 (0.09391050)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.02313694 (0.02399925), PSNR 16.35694122 (16.19989465), SSIM 0.13047083 (0.12477459)
Finished training epoch 2
Validate: MSE 0.02354968 (0.02505784), PSNR 16.28014946 (16.01658814), SSIM 0.07052466 (0.09403337)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.02387484 (0.02348763), PSNR 16.22059441 (16.29299063), SSIM 0.14130700 (0.14017237)
Finished training epoch 3
Validate: MSE 0.02348297 (0.0

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 15 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 18 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 6 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 4 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 17 pixels
  return func(*args, **kwargs)
/

Validate: MSE 0.07776409 (0.08091446), PSNR 11.09220791 (10.92056057), SSIM 0.09020375 (0.10403450)
Finished validation.
Starting training epoch 20
Epoch: 20, MSE 0.02124378 (0.02145571), PSNR 16.72768211 (16.68568227), SSIM 0.21501899 (0.21913904)
Finished training epoch 20


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return func(*args, **kwargs)


Validate: MSE 0.04285744 (0.04526695), PSNR 13.67973709 (13.44386988), SSIM 0.11632146 (0.13183658)
Finished validation.
Starting training epoch 21
Epoch: 21, MSE 0.02064737 (0.02140390), PSNR 16.85135269 (16.69627188), SSIM 0.22034602 (0.22090578)
Finished training epoch 21


...

Validate: MSE 0.06359131 (0.06741812), PSNR 11.96602249 (11.71343920), SSIM 0.11039574 (0.11753614)
Finished validation.
Starting training epoch 25
Epoch: 25, MSE 0.02162803 (0.02123721), PSNR 16.64982986 (16.73024398), SSIM 0.21920209 (0.22653627)
Finished training epoch 25
Validate: MSE 0.02475971 (0.02620583), PSNR 16.06254196 (15.81932253), SSIM 0.14908454 (0.15955350)
Finished validation.
Starting training epoch 26
Epoch: 26, MSE 0.02145757 (0.02118673), PSNR 16.68419266 (16.74034941), SSIM 0.23323087 (0.22791021)
Finished training epoch 26
Validate: MSE 0.02522242 (0.02672157), PSNR 15.98213196 (15.73515756), SSIM 0.14566338 (0.15706023)
Finished validation.
Starting training epoch 27
Epoch: 27, MSE 0.02195209 (0.02116117), PSNR 16.58524132 (16.74580970), SSIM 0.22318988 (0.22862318)
Finished training epoch 27
Validate: MSE 0.02571917 (0.02714631), PSNR 15.89743042 (15.66630563), SSIM 0.14628103 (0.15873373)
Finished validation.
Starting training epoch 28
Epoch: 28, MSE 0.0218675

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return func(*args, **kwargs)


Validate: MSE 0.02680139 (0.02779988), PSNR 15.71842480 (15.56271487), SSIM 0.10177481 (0.12297740)
Finished validation.
Starting training epoch 34
Epoch: 34, MSE 0.01986461 (0.02099993), PSNR 17.01919937 (16.77933297), SSIM 0.24995634 (0.23291496)
Finished training epoch 34
Validate: MSE 0.02731857 (0.02832825), PSNR 15.63541889 (15.48118230), SSIM 0.11470367 (0.13755907)
Finished validation.
Starting training epoch 35
Epoch: 35, MSE 0.01939572 (0.02096402), PSNR 17.12294006 (16.78637963), SSIM 0.24481606 (0.23370643)
Finished training epoch 35
Validate: MSE 0.02441471 (0.02530521), PSNR 16.12348557 (15.97256223), SSIM 0.11200586 (0.13434492)
Finished validation.
Starting training epoch 36
Epoch: 36, MSE 0.02095387 (0.02096898), PSNR 16.78735733 (16.78546072), SSIM 0.23899226 (0.23362865)
Finished training epoch 36
Validate: MSE 0.02354970 (0.02451956), PSNR 16.28014565 (16.10921543), SSIM 0.12289149 (0.14361313)
Finished validation.
Starting training epoch 37
Epoch: 37, MSE 0.0204040

Validate: MSE 0.02436822 (0.02601947), PSNR 16.13176155 (15.85152719), SSIM 0.12051721 (0.13777072)
Finished validation.
Starting training epoch 64
Epoch: 64, MSE 0.02063333 (0.02065279), PSNR 16.85430717 (16.85140424), SSIM 0.24758562 (0.23997827)
Finished training epoch 64
Validate: MSE 0.02636672 (0.02810291), PSNR 15.78943825 (15.51634287), SSIM 0.11403664 (0.13326245)
Finished validation.
Starting training epoch 65
Epoch: 65, MSE 0.02044365 (0.02063603), PSNR 16.89441490 (16.85490718), SSIM 0.24235854 (0.24032140)
Finished training epoch 65
Validate: MSE 0.02723994 (0.02910675), PSNR 15.64793873 (15.36388304), SSIM 0.09889197 (0.12108133)
Finished validation.
Starting training epoch 66
Epoch: 66, MSE 0.02081939 (0.02064484), PSNR 16.81532097 (16.85306904), SSIM 0.24339195 (0.24000659)
Finished training epoch 66
Validate: MSE 0.02361908 (0.02528192), PSNR 16.26736832 (15.97695617), SSIM 0.10243014 (0.12296530)
Finished validation.
Starting training epoch 67
Epoch: 67, MSE 0.0204008

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.02514447 (0.02680323), PSNR 15.99557495 (15.72238916), SSIM 0.09533089 (0.11859731)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()